In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install transformers
!pip install pytorch_transformers
!pip install -r drive/MyDrive/transformers_cl/requirements.txt

In [ ]:
# Please install apex from https://www.github.com/nvidia/apex to use fp16 training.
# !git clone https://github.com/NVIDIA/apex
# !cd apex
# !pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

# packages needed for running other scoring_functions
# !pip install fasttext
# !pip install gensim==3.8.3 

# this is for calculating other difficulty criterion scores
# !python drive/MyDrive/transformers_cl/calculate_scoring_function.py \
# --dataset_path drive/MyDrive/MSDialog/train.tsv \
# --output_path drive/MyDrive/MSDialog/

In [ ]:
# after fine-tuning the BERT model and evaluate on the training dataset
# we can get a file containing predictions of labels
# for each training sample, we have one prediction for its relevant answer
# and one prediction for its irrelevant answer 
# this code is calculating the difference between them 
# and for generating scoring files based on bert predictions
import pandas as pd
seed = 42
common_path = "drive/MyDrive/transformers_cl"
task_name = "ms_v2"
model_name = "bert" 
end = ["6"]
for i in end:
  path = common_path + '/' + task_name + '_' + model_name + '_' + str(i)
  df = pd.read_csv(path+"/preds_run_cl__seed_"+str(seed), names=["preds_run_"+str(seed)])
  difs_df = []
  for i in range(len(df)):
    if i % 2 != 0:
      difs_df.append(df.values[i-1][0]-df.values[i][0])
  difs_df = pd.DataFrame(difs_df, columns = ["preds_dif_run_"+str(seed)])
  difs_df.to_csv(path + "/preds_dif_c_values_3", header=False, index=False)		

In [ ]:
def read_scores_file(path):    
    with open(path,'r') as f:
        scores = [float(l.split("\n")[0]) for l in f.readlines()]
    return scores

# Fine-tuning logs for generating prediction files

# -----------------------------------------------------------Mantis----------------------------------------------------

# 08-05 batch size = 8, logging_step = 1000, trained epoch = 1, map = 0.93

In [ ]:
# have generated 9 ns with bm25 sampler
# added fp16 to save memory, not sure if helpful, removed because don't bother making it work
# 8 hours 0.7 epoch
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

08/05/2021 09:22:22 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/05/2021 09:22:22 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpdqhhvmm_
100% 433/433 [00:00<00:00, 557267.15B/s]
08/05/2021 09:22:22 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpdqhhvmm_ to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/05/2021 09:22:22 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/05/2021 09:22:22 - INFO - pytorch_transformers.file_u

In [ ]:
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

08/06/2021 06:22:47 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/06/2021 06:22:47 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp1_a9f6uj
100% 433/433 [00:00<00:00, 391070.98B/s]
08/06/2021 06:22:47 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp1_a9f6uj to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/06/2021 06:22:47 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/06/2021 06:22:47 - INFO - pytorch_transformers.file_u

# 08-05 batch size = 32, logging_step = 1000, trained epoch = 1, map = 0.93

In [ ]:
# have generated 9 ns with bm25 sampler
# added fp16 to save memory, not sure if helpful, removed because don't bother making it work
# 3h30m
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=32   \
    --per_gpu_train_batch_size=32   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_bm25_3/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

08/05/2021 11:44:08 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/05/2021 11:44:08 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpa8b04wor
100% 433/433 [00:00<00:00, 381701.06B/s]
08/05/2021 11:44:09 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpa8b04wor to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/05/2021 11:44:09 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/05/2021 11:44:09 - INFO - pytorch_transformers.file_u

# 08-05 batch size = 64, logging_step = 1000, trained epoch = 1, map = 0.92

In [ ]:
# have generated 9 ns with bm25 sampler
# 3h3m
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_bm25_64/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

08/05/2021 10:03:42 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/05/2021 10:03:42 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp8rmq56su
100% 433/433 [00:00<00:00, 422454.90B/s]
08/05/2021 10:03:43 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp8rmq56su to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/05/2021 10:03:43 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/05/2021 10:03:43 - INFO - pytorch_transformers.file_u

# --------------------------failed trials-----------------------------

In [ ]:
# have generated 9 ns with random sampler, try again     
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers/mantis_random_9 \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_valid_random/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

08/04/2021 16:52:07 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/04/2021 16:52:08 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpud79atmm
100% 433/433 [00:00<00:00, 389561.05B/s]
08/04/2021 16:52:08 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpud79atmm to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/04/2021 16:52:08 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/04/2021 16:52:08 - INFO - pytorch_transformers.file_u

In [ ]:
import pandas as pd
valid = pd.read_csv("drive/MyDrive/transformer_rankers/valid.tsv", sep="\t",  error_bad_lines=False)
valid.size//3

180959

In [ ]:
# tested on a new negative sampling valid, with 9 ns in xiaokang's drive
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_test/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

In [ ]:
# tested on a new negative sampling valid, which has no responses from the train dataset, but the map is still very high
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_8test/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

In [ ]:
# because the negative sampling is bert-base-cased, I want to check if any different things can happen??? 
# But the result is no, almost the same 
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-cased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_8test_cased/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

# 07-27 batch size = 8, trained epoch = 1, map very high, sentenceBert negative sampling

In [ ]:
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_83/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

08/01/2021 07:10:26 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/01/2021 07:10:27 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/01/2021 07:10:27 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "mantis_10",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,

# 07-27 batch size = 64, trained epoch = 1, map = , sentenceBert negative sampling

In [ ]:
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/transformer_rankers \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_364/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 100 \

07/27/2021 06:37:05 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/27/2021 06:37:06 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
07/27/2021 06:37:06 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "mantis_10",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,

# 07-19 batch size = 8, trained epoch = 1, map very high, random negative sampling

In [ ]:
# running time 7h15m for 1 epochs random negative sampling
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_83/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

07/19/2021 13:39:59 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/19/2021 13:40:00 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
07/19/2021 13:40:00 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "mantis_10",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,

In [ ]:
# 7个小时白跑了
# 后来耍了点花招，其实也不是啦，是preds那边忘记改了，只有最后1/4的数据参与了最后一个compute_metric的计算
# 总之scoring file没问题啦
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_83/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

07/23/2021 07:50:19 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/23/2021 07:50:19 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
07/23/2021 07:50:19 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "mantis_10",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,

In [ ]:
a = (1,2)
len(a)

2

# 07-19 batch size = 64, trained epoch = 1, map very high, random negative sampling

In [ ]:
# running time：2h5m  for 3 epochs 
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_364/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 100 \

07/19/2021 21:19:29 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/19/2021 21:19:29 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp9j85fg5t
100% 433/433 [00:00<00:00, 470866.90B/s]
07/19/2021 21:19:29 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp9j85fg5t to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
07/19/2021 21:19:29 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
07/19/2021 21:19:29 - INFO - pytorch_transformers.file_u

In [ ]:
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --output_dir drive/MyDrive/transformers_cl/mantis_10_bert_364/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

07/23/2021 07:52:27 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/23/2021 07:52:27 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpiel50oi9
100% 433/433 [00:00<00:00, 474434.07B/s]
07/23/2021 07:52:27 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpiel50oi9 to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
07/23/2021 07:52:27 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
07/23/2021 07:52:27 - INFO - pytorch_transformers.file_u

# ------------------------------------------------------------MSDialog----------------------------------------------

# 09-07 batch size = 8, max_seq_length = 256

In [ ]:
# running time 2h30m for 1 epochs 
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 256 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_6/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

09/07/2021 11:28:12 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
09/07/2021 11:28:12 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
09/07/2021 11:28:12 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "ms_v2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "

In [ ]:
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 256 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_6/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

09/07/2021 14:02:05 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
09/07/2021 14:02:05 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
09/07/2021 14:02:05 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "ms_v2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "

# 09-07 batch_size = 8, max_seq_length = 512

In [ ]:
# running time 4h30m for less than 1 epoch 
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 512 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_6/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

09/07/2021 07:16:21 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
09/07/2021 07:16:22 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
09/07/2021 07:16:22 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "ms_v2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "

In [ ]:
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 512 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_6/ \
    --save_aps \
    --eval_all_checkpoints \

09/07/2021 12:02:33 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
09/07/2021 12:02:33 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpqmwxadmh
100% 433/433 [00:00<00:00, 316565.04B/s]
09/07/2021 12:02:34 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpqmwxadmh to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
09/07/2021 12:02:34 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
09/07/2021 12:02:34 - INFO - pytorch_transformers.file_u

# 05-18 batch size = 8, trained epoch = 3, map = 0.95

In [ ]:
# running time 4h15m for 3 epochs 
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_5/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 1000 \

05/18/2021 07:54:58 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/18/2021 07:54:58 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
05/18/2021 07:54:58 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "ms_v2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "

# 06-11 batch size = 64, trained epoch = 3, map = 0.97

In [ ]:
# running time：2h5m  for 3 epochs 
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_3/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 100 \

06/11/2021 10:56:02 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/11/2021 10:56:02 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp_h22ae7k
100% 433/433 [00:00<00:00, 439741.80B/s]
06/11/2021 10:56:02 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp_h22ae7k to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
06/11/2021 10:56:02 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
06/11/2021 10:56:02 - INFO - pytorch_transformers.file_u

In [ ]:
# import fasttext.util
# fasttext.util.download_model('en', if_exists='ignore')  # English
# ft = fasttext.load_model('cc.en.300.bin')
# ft.save_model('cc.en.100.vec')

# from gensim.models import KeyedVectors
# wvecs = KeyedVectors.load_word2vec_format('cc.en.300.bin',
#         binary=True) 
# # The following way doesn't work
# from gensim.models.fasttext import FastText
# mod = FastText.load_fasttext_format('cc.en.300.bin')
# from gensim.models.fasttext import FastText as FastText_gensim
# pretrained_model=FastText_gensim.load('cc.en.300.bin')

# 06-14 batch size = 64, use untrained model, map = 0.72

In [ ]:
# I want to use the untrained model so do_train = False
!python drive/MyDrive/transformers_cl/run_glue_copy.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert \
    --eval_all_checkpoints \
    --logging_steps 100 \
    --save_aps \

06/14/2021 12:35:28 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/14/2021 12:35:28 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
06/14/2021 12:35:28 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "ms_v2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "

# 06-16 batch size = 64, trained epoch = 1, map = 0.95

In [ ]:
# running time：45m, for 1 epoch
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_1/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 100 \

06/16/2021 06:55:49 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/16/2021 06:55:49 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
06/16/2021 06:55:49 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "ms_v2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "

# 06-16 batch size = 64, trained epoch = 2, map = 0.96

In [ ]:
# running time：2h5m  for 2 epochs 
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 2 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_2/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 100 \

06/16/2021 07:10:39 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/16/2021 07:10:39 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmppbms0bwm
100% 433/433 [00:00<00:00, 494994.18B/s]
06/16/2021 07:10:39 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmppbms0bwm to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
06/16/2021 07:10:39 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
06/16/2021 07:10:39 - INFO - pytorch_transformers.file_u

# 06-16 batch size = 64, trained epoch = 4, map = 0.98

In [ ]:
# running time：3h  for 4 epochs 
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 4 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_4/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 100 \

06/16/2021 07:52:55 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/16/2021 07:52:55 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
06/16/2021 07:52:55 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "ms_v2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "

In [ ]:
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  ms_v2 \
    --do_eval \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/MSDialog \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --output_dir drive/MyDrive/transformers_cl/ms_v2_bert_4/ \
    --save_aps \
    --eval_all_checkpoints \
    --logging_steps 100 \

06/16/2021 10:52:49 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/16/2021 10:52:50 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpf8bp698j
100% 433/433 [00:00<00:00, 355129.77B/s]
06/16/2021 10:52:50 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpf8bp698j to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
06/16/2021 10:52:50 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
06/16/2021 10:52:50 - INFO - pytorch_transformers.file_u